In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [5]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import * # import NumericalFunctions 

In [6]:
####################################
#LOADING CLASSES

In [7]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="LagrangianArrays", dataName="Lagrangian_Binary_Array",
                                dtype='int16')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DC

In [15]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=30
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

Running timesteps from 0:133 



In [9]:
####################################
#FUNCTIONS

In [10]:
def GetZYX(inputParcelDirectory, timeString):
    #Lagrangian Position Arrays
    ##############
    def grid_location(z,y,x):
        zf=ModelData.zf*1000; which_zh=np.clip(np.searchsorted(zf,z)-1,0,None).astype(np.uint16)
        #which_zh=np.where(which_zh == -1, 0, which_zh) 
        
        yf=ModelData.yf*1000; which_yh=np.clip(np.searchsorted(yf,y)-1,0,None).astype(np.uint16) 
        #which_yh=np.where(which_yh == -1, 0, which_yh) 
        
        xf=ModelData.xf*1000; which_xh=np.clip(np.searchsorted(xf,x)-1,0,None).astype(np.uint16)
        #which_xh=np.where(which_xh == -1, 0, which_xh) 
        
        return which_zh,which_yh,which_xh
    
    # print('Creating Lagrangian X,Y,Z Binary Arrays')
    x = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="x")
    y = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="y")
    z = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="z")
    Z,Y,X=grid_location(z,y,x)
    return z,x,Z,Y,X

def GetUW(inputParcelDirectory, timeString):
    #GETTING u and w for Lagrangian_Tracking later
    u = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="u")
    w = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="w")
    return u,w

In [11]:
def MakeLagrangianArray(inputDataDirectory, timeString,varnames,Z,Y,X): #FAST ADVANCED INDEXING VERSION
    # print('Making Variables Lagrangian Array')
    # Initialize dictionaries
    var_data_dict = {varname: CallVariable(ModelData, DataManager, timeString, varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    # print('Running')
    # Nt = len(data['time'])
    # Get Indices
    z_idx = Z
    y_idx = Y
    x_idx = X
    # ts = np.arange(Nt)[:, None]
    # t_idx = np.broadcast_to(ts, Z.shape)

    for varname, var_data in var_data_dict.items():
        # VAR[varname] = var_data[t_idx, z_idx, y_idx, x_idx]
        VAR[varname] = var_data[z_idx, y_idx, x_idx]

    return [VAR[varname] for varname in varnames]
    # return VAR

In [12]:
def GetVariables(inputDataDirectory,timeString, Z,Y,X):
    # print('Making W and QCQI Lagrangian Array')
    varnames=['winterp','qc','qi']
    [W,QC,QI] = MakeLagrangianArray(inputDataDirectory,timeString, varnames, Z,Y,X)
    QCQI=QC+QI
    return W,QC,QCQI
    
# def GetBinaryArrays(inputDataDirectory,timeString, W,QCQI): #CALCULATED HERE
#     def GetCloudyUpdraftThresholds():
#         w_thresh1=0.1
#         w_thresh2=0.5
#         qcqi_thresh=1e-6
#         return w_thresh1,w_thresh2,qcqi_thresh

#     #Create Set Thresholds and Create Binary Arrays
#     w_thresh1,w_thresh2,qcqi_thresh = GetCloudyUpdraftThresholds()
    
#     #Apply Thresholds 
#     print('Making Lagrangian Binary Arrays')
#     print('Getting A_g')
#     A_g = np.where((W >= w_thresh1) & (QCQI < qcqi_thresh), True, False)
#     print('Making A_c')
#     A_c = np.where((W >= w_thresh2) & (QCQI >= qcqi_thresh), True, False)
#     print('done')
    
#     outputDictionary = {"A_g": A_g,
#                         "A_c": A_c
#                        }
#     return outputDictionary

def GetBinaryArrays(DataManager,timeString, z,x,Z,Y,X): #PRECALCULATED
    # print('Making W and QCQI Lagrangian Array')
    varnames=['A_g','A_c']
    [A_g,A_c] = MakeLagrangianArray(DataManager,timeString, varnames, Z,Y,X)

    outputDictionary = {"A_g": A_g,
                        "A_c": A_c,
                        
                        "z": z,
                        "x": x,
                        "Z": Z,
                        "Y": Y,
                        "X": X
                       }
    return outputDictionary

In [13]:
###############################################################
#RUNNING

In [16]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    z,x,Z,Y,X = GetZYX(DataManager.inputParcelDirectory, timeString)
    [W,QC,QCQI] = GetVariables(DataManager.inputDataDirectory,timeString, Z,Y,X)

    #calculating variables
    outputDictionary = GetBinaryArrays(DataManager,timeString, z,x,Z,Y,X)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary)

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/Lagrangian_Binary_Array/Lagrangian_Binary_Array_1km_5min_34nz_0-00-00.h5 

Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/Lagrangian_Binary_Array/Lagrangian_Binary_Array_1km_5min_34nz_0-05-00.h5 

Current time 2
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/Lagrangian_Binary_Array/Lagrangian_Binary_Array_1km_5min_34nz_0-10-00.h5 

Current time 3
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/Lagrangian_Binary_Array/Lagrangian_Binary_Array

In [ ]:
######################################################

In [15]:
# #READING BACK IN
# def GetVariableData(t, dataType="Lagrangian_Binary_Array", dataName="Lagrangian_Binary_Array"):
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataType)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)

Keys in file: ['A_c', 'A_g']
A_c: shape = (1000000,), dtype = float32
A_g: shape = (1000000,), dtype = float32
